In [16]:
!pip install sentence-transformers
!pip install -U dspy-ai



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
  Using cached dspy_ai-3.1.3-py3-none-any.whl.metadata (285 bytes)
  Using cached dspy-3.1.3-py3-none-any.whl.metadata (8.4 kB)
Using cached dspy_ai-3.1.3-py3-none-any.whl (1.1 kB)
Using cached dspy-3.1.3-py3-none-any.whl (312 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [dspy-ai]

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import importlib.metadata
print(importlib.metadata.version('dspy-ai'))

3.1.3


In [18]:
pip install  datasets


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
!pip install "dspy-ai[chromadb]"



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import dspy
from datasets import load_dataset
import os
import json
from dotenv import load_dotenv
from openai import OpenAI

In [3]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

MODEL = "gpt-4o-mini"
openai = OpenAI()


OpenAI API Key exists and begins sk-proj-


In [4]:
lm = dspy.LM('ollama_chat/llama3', api_base='http://localhost:11434', api_key='')
#lm = dspy.LM('openai/gpt-4o-mini')

In [5]:
import chromadb

Define the embedding function

In [6]:
from chromadb.utils import embedding_functions
# Native Chroma way to load the same model
ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

Define the retriever class

In [9]:
class LocalChromaRetriever(dspy.Retrieve):
    def __init__(self, collection_name, persist_path, ef, k=3):
        super().__init__(k=k) # Properly initialize the base class
        self.client = chromadb.PersistentClient(path=persist_path)
        self.collection = self.client.get_or_create_collection(
            name=collection_name, embedding_function=ef
        )
        self.k = k

    def forward(self, query_or_queries, k=None):
        queries = [query_or_queries] if isinstance(query_or_queries, str) else query_or_queries
        k = k if k is not None else self.k

        results = self.collection.query(query_texts=queries, n_results=k)

        # DSPy 3.1.3 requires each passage to have a 'long_text' attribute
        passages = [
            dspy.Prediction(
                long_text=doc, 
                id=id_, 
                metadata=meta
            )
            for docs, ids, metas in zip(results["documents"], results["ids"], results["metadatas"])
            for doc, id_, meta in zip(docs, ids, metas)
        ]

        return passages # Return the list directly; dspy.Retrieve wraps this in a Prediction


In [10]:
# Define the signature for generating the final answer
class GenerateAnswer(dspy.Signature):
    """Answer the question based on the provided context."""
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    response = dspy.OutputField(desc="often between 1 and 8 words")
    

In [30]:
# 3. Build the RAG Module
class RAG(dspy.Module):
    def __init__(self, num_passages=3,retriever=None):
        super().__init__()
        # Use the retriever you configured 
        self.retriever = retriever or dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        # 1. Use the context manager to lock in your specific retriever
        with dspy.settings.context(rm=self.retriever):
            # 2. In DSPy 3.x, dspy.Retrieve(k=3)(question) returns an object with a .passages attribute
            results = dspy.Retrieve(k=3)(question)
            context = results.passages

        # 3. Generate the answer using your COT predictor
        prediction = self.generate_answer(context=context, question=question)
        
        # 4. Return a Prediction object. 
        # Match 'response' to what your metric (semantic_f1) expects.
        return dspy.Prediction(context=context, response=prediction.answer)


**Load HotPot Data**

In [31]:
# Load the dataset
hotpot = load_dataset("hotpot_qa", "fullwiki")

# Filter ONLY the train split for bridge and comparision
multihop_train = hotpot['train'].filter(lambda x: x["type"] in ["bridge", "comparison"])

singlehop_ds = hotpot['train'].filter(lambda x: x["type"] not in ["bridge", "comparison"])

print(f"Multi-hop count: {len(multihop_train)}")


# Inspect one example
print(multihop_train[0])


Multi-hop count: 90447
{'id': '5a7a06935542990198eaf050', 'question': "Which magazine was started first Arthur's Magazine or First for Women?", 'answer': "Arthur's Magazine", 'type': 'comparison', 'level': 'medium', 'supporting_facts': {'title': ["Arthur's Magazine", 'First for Women'], 'sent_id': [0, 0]}, 'context': {'title': ['Radio City (Indian radio station)', 'History of Albanian football', 'Echosmith', "Women's colleges in the Southern United States", 'First Arthur County Courthouse and Jail', "Arthur's Magazine", '2014–15 Ukrainian Hockey Championship', 'First for Women', 'Freeway Complex Fire', 'William Rast'], 'sentences': [["Radio City is India's first private FM radio station and was started on 3 July 2001.", ' It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).', ' It plays Hindi, English and regional songs.', ' It was launched in Hyderabad in March 2

In [32]:
# Build DSPy Examples with titles preserved
examples = []

# Just take the first 100 "True" Multi-hop questions
multihop_train_subset = multihop_train.select(range(100))

# Hugging Face datasets allow direct slicing to get a list of dicts 
# or you can select a range:
for ex in multihop_train_subset:   
    
    # each row is a dict
   
    titles = ex["context"]["title"]
    sentences_list = ex["context"]["sentences"]
    
    # Note: In the 'hotpot_qa' dataset specifically, 
    # context is often a dict: {"title": [...], "sentences": [[...], [...]]}
    # We need to zip them together.
    docs = [f"Title: {t}. {' '.join(s)}" for t, s in zip(titles, sentences_list)]

    example = dspy.Example(
        question=ex["question"],
        response=ex["answer"],
        context=docs
    ).with_inputs("question")

    examples.append(example)

    
print("Built", len(examples), "examples")
print("Sample Example:\n", examples[0])


Built 100 examples
Sample Example:
 Example({'question': "Which magazine was started first Arthur's Magazine or First for Women?", 'response': "Arthur's Magazine", 'context': ["Title: Radio City (Indian radio station). Radio City is India's first private FM radio station and was started on 3 July 2001.  It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).  It plays Hindi, English and regional songs.  It was launched in Hyderabad in March 2006, in Chennai on 7 July 2006 and in Visakhapatnam October 2007.  Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that offers music related news, videos, songs, and other music-related features.  The Radio station currently plays a mix of Hindi and Regional music.  Abraham Thomas is the CEO of the company.", "Title: History of Albanian football. Football in Albania e

In [24]:

# Setup persistent storage
client = chromadb.PersistentClient(path="./hotpot_chroma")
# Define the exact same embedding function used in your retriever
ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

#  Create collection (Chroma uses Sentence Transformers by default)
try:
    client.delete_collection(name="hotpot_v1")
except:
    pass # Collection didn't exist, that's fine
collection = client.get_or_create_collection(
    name="hotpot_v1", 
    embedding_function=ef
)


# 1. Collect all docs from your 100 examples (approx 1,000 docs)
docs_to_index = {}
for ex in examples:
    for doc_text in ex.context:
        docs_to_index[doc_text] = doc_text 

small_subset_texts = list(docs_to_index.values())

# 2. Ingest ONLY this small subset (< 5 seconds on M4 Mini)
collection.add(
    documents=small_subset_texts,
    ids=[f"hp_small_{i}" for i in range(len(small_subset_texts))]
)

print(f"Ingested {len(small_subset_texts)} docs. You can now test your RAG!")

Ingested 992 docs. You can now test your RAG!


**Create a  retriever instance pointing to the   database**

In [25]:
# Create a  retriever instance pointing to the   database
hotpot_rm = LocalChromaRetriever(
    collection_name="hotpot_v1", 
    persist_path="./hotpot_chroma", 
    ef=ef
)
dspy.settings.configure(lm=lm, rm=hotpot_rm)
dspy.configure(adapter=dspy.JSONAdapter())

This is a multi hop RAG class

In [26]:
class GenerateSearchQuery(dspy.Signature):
    """
    Analyze the question and context. If you find an alias or stage name or  alternative identity 
    for the entity, your next query MUST use that specific alias to find the target.
    """
    context = dspy.InputField(desc="Gathered facts, including potential aliases.")
    question = dspy.InputField()
    history = dspy.InputField(desc="Previous queries already attempted.")
    query = dspy.OutputField(desc="A new distinct query to find missing details, using the alias or other names if found.")

class GenerateAnswer(dspy.Signature):
    """Answer the question based strictly on the provided context."""
    context = dspy.InputField()
    question = dspy.InputField()
    answer = dspy.OutputField(desc="A concise answer.")

class ShouldStop(dspy.Signature):
    """
    Evaluate if the gathered context contains the specific answer to the question.
    Only return True if the EXACT attribute requested (e.g., nationality, place, date, name) 
    is explicitly mentioned in the context. Otherwise, return False.
    """
    question = dspy.InputField()
    context = dspy.InputField()
    is_sufficient = dspy.OutputField(desc="True or False", type=bool)


class SimplifiedBaleen(dspy.Module):
    def __init__(self, retriever, passages_per_hop=4, max_hops=3):
        super().__init__()
        # Store the hotpot_rm instance you passed in
        self.retriever = retriever
        self.max_hops = max_hops
        self.should_stop = dspy.Predict(ShouldStop) # Fast predictor for the stop check
        # dspy.Retrieve uses the rm from dspy.settings or the one we provide
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        
        # Replace manual f-strings with COT Predictors
        self.generate_query = dspy.ChainOfThought(GenerateSearchQuery)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = []
        history = []

        # --- NEW: Initial Discovery Hop ---
        # Search for the literal string first to find the real Alias
        with dspy.settings.context(rm=self.retriever):
            initial_psgs = self.retrieve(question).passages
            context = list(dict.fromkeys(initial_psgs))
            
        for hop in range(self.max_hops):

            #early stopping    
            if context:
                stop_pred = self.should_stop(question=question, context=context)
                if "true" in str(stop_pred.is_sufficient).lower():
                    break
                    
            # 1. Generate a query based on current context
            query_pred = self.generate_query(context=context, question=question,history=history)
            history.append(query_pred.query) # Keep track of what we've searched
            
            # 2. Use the specific hotpot_rm via context manager
            with dspy.settings.context(rm=self.retriever):
                new_passages = self.retrieve(query_pred.query).passages
            
            # 3. Deduplicate
            context = list(dict.fromkeys(context + new_passages))
            
        # 4. Generate final answer
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, response=prediction.answer)



In [40]:
rag_baleen = SimplifiedBaleen(retriever=hotpot_rm)
rag_single=RAG(retriever=hotpot_rm)
from dspy.evaluate import metrics

In [28]:
dspy.settings.lm.history = []
# Assume you already built `examples` from HotpotQA
sample = examples[4]   # pick the first example, or any index

print("Question:", sample.question)
print("Gold Answer:", sample.response)

# Run it through your retriever + Baleen pipeline
prediction = rag_baleen(sample.question)

print("\nPredicted Answer:", prediction.response)
print("Retrieved Context (first 3 docs):")

    
for doc in prediction.context[:3]:
    # Check if doc is a dict (standard in DSPy 3.x) or a string
    text = doc.get('text', doc.get('long_text', str(doc))) if isinstance(doc, dict) else doc
    print("-", text[:200], "...")



Question: Cadmium Chloride is slightly soluble in this chemical, it is also called what?
Gold Answer: alcohol

Predicted Answer: alcohol
Retrieved Context (first 3 docs):
- Title: Cadmium chloride. Cadmium chloride is a white crystalline compound of cadmium and chlorine, with the formula CdCl.  It is a hygroscopic solid that is highly soluble in water and slightly solubl ...
- Title: Chloride. The chloride ion is the anion (negatively charged ion) Cl.  It is formed when the element chlorine (a halogen) gains an electron or when a compound such as hydrogen chloride is dissol ...
- Title: Gold(III) chloride. Gold(III) chloride, traditionally called auric chloride, is a chemical compound of gold and chlorine.  With the molecular formula AuCl, the name gold trichloride is a simpli ...


In [19]:
for i, h in enumerate(dspy.settings.lm.history):
    print(f"--- Prompt {i} ---")
    print("Prompt:", h.get("prompt"))
    print("Raw Outputs:", h.get("outputs"))


--- Prompt 0 ---
Prompt: None
Raw Outputs: ['{\n"is_sufficient": "True"\n}']
--- Prompt 1 ---
Prompt: None
Raw Outputs: ['{\n"reasoning": "The given context discusses Cadmium chloride\'s properties and solubility. Since it is mentioned as being \'slightly soluble\' in alcohol, we can infer that the chemical it is slightly soluble in is also present in the same passage.",\n"answer": "alcohol"\n}']


In [37]:

class SemanticF1Signature(dspy.Signature):
    """Assess the semantic similarity between a predicted answer and a gold answer."""
    prediction = dspy.InputField()
    gold_answer = dspy.InputField()
    score = dspy.OutputField(desc="A float between 0.0 and 1.0")

def semantic_f1(example, pred, trace=None):
    # This uses your default dspy.settings.lm to judge the similarity
    judge = dspy.Predict(SemanticF1Signature)
    
    # Map the fields: example.answer is the gold, pred.response is your RAG output
    result = judge(prediction=pred.response, gold_answer=example.response)
    
    try:
        return float(result.score)
    except:
        return 0.0


In [41]:

# Take a small slice of HotpotQA validation set
subset = multihop_train_subset.select(range(15))   # 5 examples for demo

scores_single = []
scores_multi = []

for ex in subset:
    # Wrap into DSPy Example
    example = dspy.Example(
        question=ex["question"],
        response=ex["answer"]
    ).with_inputs("question")

    # Run single-hop RAG
    pred_single = rag_single(example.question)
    # semantic_f1 is a callable function
    score_single = semantic_f1(example, pred_single)
    scores_single.append(score_single)

    # Run multi-hop RAG
    pred_multi = rag_baleen(example.question)
    score_multi =semantic_f1(example, pred_multi)
    scores_multi.append(score_multi)

print("Average SemanticF1 (Single-Hop RAG):", sum(scores_single)/len(scores_single))
print("Average SemanticF1 (Multi-Hop RAG):", sum(scores_multi)/len(scores_multi))


Average SemanticF1 (Single-Hop RAG): 0.6357142857142858
Average SemanticF1 (Multi-Hop RAG): 0.8357142857142857


Let us see the prompts for the last few results

In [ ]:
dspy.settings.lm.inspect_history(n=20)

In [65]:
from dspy.teleprompt import BootstrapFewShot

# 1. Define the Optimizer
# 'max_bootstrapped_demos' tells DSPy to find 2 perfect examples to put in the prompt
optimizer = BootstrapFewShot(metric=metric, max_bootstrapped_demos=2)

# 2. Compile the Baleen Module
# It's 'training' the Librarian to search better and the Researcher to answer shorter.
compiled_baleen = optimizer.compile(rag_multi, trainset=examples[:20])

 10%|███████████████████████▍                                                                                                                                                                                                                  | 2/20 [00:00<00:04,  3.85it/s]

Bootstrapped 2 full traces after 2 examples for up to 1 rounds, amounting to 2 attempts.


In [70]:
import json

# 1. Manually extract state only from the components that have 'dump_state' working
# We target the 'generate_query' and 'answer_gen' specifically
program_state = {
    "generate_query": compiled_baleen.generate_query.dump_state(),
    "answer_gen": compiled_baleen.answer_gen.dump_state()
}

# 2. Save to JSON
with open("hotpot_baleen_compiled.json", "w") as f:
    json.dump(program_state, f, indent=4)

print("Saved Predictor states manually, bypassing the Retriever bug!")


Saved Predictor states manually, bypassing the Retriever bug!
